# WISPcloud API tutorial

For accessing data collected by WISPstation instruments, [Water Insight](http://www.waterinsight.nl/ "WI's Homepage") offers an application programming interface (API). This API allows users to dynamically retrieve data from the WISPcloud database using a URL. This document is a short introduction to the WISPcloud API. Please note that this is still very much work in progress. Any feedback is appreciated!

## Accessing the API
There are several ways to access the API:
* Using a web browser. Simply type or copy the request into the address bar and the browser will display the text file.
* Using a command-line tool such as [cURL](https://curl.haxx.se// "cURL Homepage") 
* Directly in a script 

We will show examples for these options below. 

The API is password protected. As a user of a WISPstation, you should have received login credentials. There is also an open demo account:
> username: demo

> password: demo

### Access via a web browser
Simply paste the url in your browser's address bar. For a quick view, such as the ShowRecent request, using your browser is an easy and convenient way to access the API. Please note that this is not advised when retrieving larger amounts of data as browsers may at a given point stop downloading additional data without producing an error message. 

### Accessing via a command-line tool
For larger amounts of data or as part of a recurrent processing, it can be useful to download data from the API to file. This can be done using the [cURL](https://curl.haxx.se// "cURL Homepage")  tool.

```
curl -u username:password -o filename.tsv "https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-05-11T09:00,2018-05-11T19:00&INSTRUMENT=WISPstation001"
``` 

### Accessing the API directly in python
If working with the data in a script/program, the easiest way is to directly access the API from your script and read the returned data. For example, in python you could 

```python
import requests
import io

username = 'demo'
password = 'demo'
url = 'https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-05-11T09:00,2018-05-11T19:00&INSTRUMENT=WISPstation001'
datastring = io.StringIO(requests.get(url, auth=(username, password)).content.decode('utf-8')).read()
```

## Building the request URL
The request URL is built on a base URL: 
> https:wispcloud.waterinsight.nl/api/query?

followed by several key-value pairs which are built up as follows:
* values are assigned to keys by a "="
* several values for the same key are separated by ","
* key-value pairs are separated by "&"

All our requests use the service "Data". The current version is 1.0. Therefore, all request URLs will start with 
> https:wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0

followed by the specific request (see below). The keys are case-insensitive.

## The requests

### GetDocumentation

The simplest request is GetDocumentation. It returns a static text-based documentation of the API:
> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetDocumentation

### ShowRecent
The ShowRecent request returns a fixed-format file of the most recent WISPstation measurements in WISPcloud that the user has access to. 
> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=ShowRecent

The reply is a comma-separated file giving the measurement id, device name and timestamp of the 30 most recent measurements that the user has access to:
```
measurement,device,timestamp
85629,WISPstation004,2018-07-18 10:30:05.398616
85633,WISPstation001,2018-07-18 10:30:05.369909
85628,WISPstation002,2018-07-18 10:30:05.215879
85630,WISPstation005,2018-07-18 10:30:04.843113
85632,WISPstation005,2018-07-18 10:15:05.298292
85626,WISPstation004,2018-07-18 10:15:05.280844
...
```

### GetData
The GetData request is the main work horse of the WISPcloud API. It allows to flexibly retrieve data from the WISPcloud database. The reply is always a tab-separated file containing the requested data. If used without any further specifier
> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData

it will return a selected water quality parameters and measurement meta-data for the latest 100 measurements that the user has access to:
```

# HEADERLINES 18
# Request processed at 2018-07-18 10:41:16 UTC
# 
# Requested by manager from 86.91.109.85
# 
# ---original request---
# https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData
# 
# ----------------------
# Resource: https://wispcloud.waterinsight.nl/api/query
# Parameters:
#   service ['data']
#   version ['1.0']
#   request ['getdata']
# ----------------------
# 
# WARNING: No filter keywords were given in the query to keep the number of results in check. Output is capped to the most recent 100 measurements this account has access to until at least one filter is used in query.
# 
instrument.name	measurement.date	measurement.id	waterquality.tsm	waterquality.chla	waterquality.kd
	YYYY-MM-DD hh:mm:ss.sssss		g m-3	mg m-3	m-1
WISPstation004	2018-07-18 10:30:05.398616	85629	4.4	6.6	3.8
WISPstation001	2018-07-18 10:30:05.369909	85633	12.0	11.5	0.7
WISPstation002	2018-07-18 10:30:05.215879	85628	41.8	None	9.2
WISPstation005	2018-07-18 10:30:04.843113	85630	17.1	27.4	2.2
WISPstation005	2018-07-18 10:15:05.298292	85632	17.0	27.2	2.2
WISPstation004	2018-07-18 10:15:05.280844	85626	4.1	8.7	2.4
WISPstation001	2018-07-18 10:15:05.215745	85627	11.9	11.2	0.7
WISPstation002	2018-07-18 10:15:04.841156	85625	10.1	11.2	1.6
...
```

As can be seen, the return format is a tab-separated text file. A variable-length header is indicated with lines starting with "#". The first header line indicates the length of the header. The next line after the file header contains the column names for the result columns, the line after that units and other meta-data necessary for interpretation of the results. After that follows one line per measurement that the request retrieved. Spectra are given in one column in brackets. Missing data are indicated by None.

#### Selecting by date, location and instrument
To specify which data to retrieve, the query can be narrowed by date/time, location and/or instrument name.

##### Time
To restrict the query to a certain date/time range, use the TIME keyword, specifying a time interval by two ISO timestamps separated by a comma. All times are in UTC.

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-05-11T09:00,2018-05-11T19:00

If the TIME keyword is provided more than once, results matching EITHER range are returned, e.g. adding these parameters will select measurements from between 9:00 and 19:00 on the 11th and 16th of May 2018.

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-05-11T09:00,2018-05-11T19:00&TIME=2018-05-16T09:00,2018-05-16T19:00

##### Instrument

To retrieve data only from one or more specific instrument(s), use the INSTRUMENT keyword. If requesting more than on instrument, separate the instruments by ",".

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-05-11T09:00,2018-05-11T19:00&INSTRUMENT=WISPstation001

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&INSTRUMENT=WISPstation001,WISPstation005

##### Location
To select data within a certain geographic area, use the BBOX keyword. The format for the argument is BBOX=<west edge longitude>, <south edge latitude>, <east edge longitude>, <north edge latitude>

If the BBOX parameter is specified more than once, records from either area will be included. Uniqueness of records in the output set is preserved, so when areas overlap, records in the overlap area are only added to the result set once.

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&BBOX=12.0,43.0,12.2,43.2

#### Selecting specific output columns
To specify which information per measurement the results should include, use the INCLUDE keyword. When using the INCLUDE keyword, list all the columns you want in the output. Standard columns you will want to include are measurement.id and measurement.date (which includes a UTC timestamp). If you request a column that does not exist, the result will return a column of that name with all nodata (None) values.

##### Retrieving water quality parameters
As the previous examples already showed, it is possible to retrieve water quality parameters derived from the spectral measurements. The parameters that are available are with their respective column names are:
- Chlorophyll-a concentration (based on Gons 1999): waterquality.chla
- Total supsended matter (or suspended particulate matter) concentration (based on Rijkeboer 1999): waterquality.tsm
- Vertical diffuse attenuation coefficient (based on Gons 1998): waterquality.kd
- C-phycocyanin concentration (based on Simis 2005): waterquality.cpc

A return file will look like this:
```
# HEADERLINES 18
# Request processed at 2018-07-19 09:33:58 UTC
# 
# Requested by manager from 92.111.24.140
# 
# ---original request---
# https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&INCLUDE=measurement.id,measurement.date,instrument.name,waterquality.chla,waterquality.tsm,waterquality.kd,waterquality.cpc
# 
# ----------------------
# Resource: https://wispcloud.waterinsight.nl/api/query
# Parameters:
#   include ['measurement.id,measurement.date,instrument.name,waterquality.chla,waterquality.tsm,waterquality.kd,waterquality.cpc']
#   request ['getdata']
#   time ['2018-07-11t09:00,2018-07-11t19:00']
#   version ['1.0']
#   service ['data']
# ----------------------
# 
measurement.id	measurement.date	instrument.name	waterquality.chla	waterquality.tsm	waterquality.kd	waterquality.cpc
	YYYY-MM-DD hh:mm:ss.sssss		mg m-3	g m-3	m-1	mg m-3
84010	2018-07-11 09:00:05.333859	WISPstation003	None	77.5	None	54.3
84011	2018-07-11 09:00:05.176993	WISPstation002	8.1	11.7	1.6	7.4
84013	2018-07-11 09:00:05.346611	WISPstation001	7.7	14.1	1.0	None
84014	2018-07-11 09:15:05.083525	WISPstation003	None	102.4	None	63.9
84015	2018-07-11 09:15:05.677694	WISPstation002	7.1	8.8	0.9	2.5
84017	2018-07-11 09:15:05.719169	WISPstation005	20.5	14.2	2.3	28.0
84019	2018-07-11 09:15:05.244534	WISPstation001	None	34.2	6.4	None
...
```

Currently only one algorithm is implemented per parameter. It is planned to provide different algorithms to accomodate for different water types. The syntax will then be waterquality.chla.gons for the algorithm of Gons.
> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&INCLUDE=measurement.id,measurement.date,instrument.name,waterquality.chla,waterquality.tsm,waterquality.kd,waterquality.cpc

##### Retrieving spectral data

> Please note that queries that return spectral data can result in rather large files. When testing please limit the number of rows you retrieve.

The WISPstation measures downwelling irradiance, downwelling radiance and upwelling radiance. From these measurements, remote sensing reflectance is derived. The WISPstation measures in two directions, one of these directions is selected based on the time of day and instrument viewing azimuth angle (which ideally should be north, but can be different according to local constraints). So when requesting the radiance, irradiance and reflectance spectra, the query will yield the measurements of the selected channel. The following 
- Downwelling irradiance: ed.irradiance
- Downwelling radiance: ld.radiance
- Upwelling radiance: lu.radiance
- Remote sensing reflectance: level2.reflectance
The remote sensing reflectance is calculated from the irradiance and radiance measurements using a fixed rho of 0.028

> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&INCLUDE=measurement.id,measurement.date,instrument.name,ed.irradiance,ld.radiance,lu.radiance,level2.reflectance

A resulting file will look like this:
```
# HEADERLINES 18
# Request processed at 2018-07-19 09:40:16 UTC
# 
# Requested by manager from 92.111.24.140
# 
# ---original request---
# https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=GetData&TIME=2018-07-11T09:00,2018-07-11T19:00&INCLUDE=measurement.id,measurement.date,instrument.name,ed.irradiance,ld.radiance,lu.radiance,level2.reflectance
# 
# ----------------------
# Resource: https://wispcloud.waterinsight.nl/api/query
# Parameters:
#   include ['measurement.id,measurement.date,instrument.name,ed.irradiance,ld.radiance,lu.radiance,level2.reflectance']
#   request ['getdata']
#   time ['2018-07-11t09:00,2018-07-11t19:00']
#   version ['1.0']
#   service ['data']
# ----------------------
# 
measurement.id	measurement.date	instrument.name	ed.irradiance	ld.radiance	lu.radiance	level2.reflectance
	YYYY-MM-DD hh:mm:ss.sssss		W/(m2*nm) for wavelength [350..900] in 1nm steps	W/(m2*nm*sr) for wavelength [350..900] in 1nm steps	W/(m2*nm*sr) for wavelength [350..900] in 1nm steps	1/sr for wavelength [350..900] in 1nm steps
84010	2018-07-11 09:00:05.333859	WISPstation003	[0.06620044,0.06827728,...,0.05627904]	[0.00525995,0.00541506,...,0.00527671]	[0.00463759,0.00476673,...,0.00470537]	[0.06782901,0.06759363,...,0.08098264]
...
```

It is also possible to retrieve the measurements of specific channels, irrespective of whether or not they are the "selected" channel for the specific measurement. The channels are as follows:

- eda.irradiance: Ed at the back 'aft' top of the instrument
- edf.irradiance: Ed at the front 'fore' top of the instrument
- ldp.radiance: Ld channel at the left 'port' front of the instrument
- lds.radiance: Ld channel at the right 'starboard' front of the instrument
- lup.radiance: Lu channel at the left 'port' front of the instrument
- lus.radiance: Lu channel at the right 'starboard' front of the instrument

Please note that the radiance channels are "cross-eyed", i.e. they look across the center line of the instrument. So the right "starboar" Lu and Ld channels look at 337.5 degree and the left "port" channels at 22.5 from the instrument orientation, usually north. 

To get the full orientation, you can combine the information about the instrument's orientation as a whole with the orientation of each channel with respect to the instrument orientation. The instrument's orientation is available as site.azimuth. The channel orientations of the radiance channels as ldp.channel.orientation, lds.channel.orientation, lup.channel.orientation, lus.channel.orientation. The Ed channels are both facing straight up and therefore have an orientation of 0.

To find out which channel was selected for a specific measurement, use the keywords ed.selected, ld.selected and lu.selected.

It is important to recognize that INCLUDE parameters are only about what is displayed, not which way the calculation goes. Including explicitly the data from certain channels in your output does NOT make those the channels used in the reflectance and subsequent water quality calculations! It only means you have the option to also see what was in the channel not used in the calculation.

##### Quality flagging
Some basic quality screening is done on the spectra. Based on a number of flags, each spectrum gets a quality classification as 'okay', 'suspect' or 'invalid'. These quality indicators can be retrieved using the columns ed.quality, ld.quality, lu.quality and level2.quality.

Please note that due to very restrictive flagging, currently almost all measurements are shown as 'suspect'. At this point, please ignore this we will be checking and updating the quality flagging.

!!! Important caveat: The level2.quality column has a very specific functionality: Per default, measurements where level2.quality is 'invalid' will not be shown in the output. Adding this column not only adds a column to the output, but as explicitly requesting the quality implies that the user is aware of the possible different quality levels of data it also makes visible the records in your result set where the quality is below the standard of quality that is shown by default.


##### Additional information
Some additional information about the measurement, the instrument, and the measurement site can be requested using the following columns:
- measurement.date: Date and time of the measurement
- measurement.id: Unique identifier for this measurement
- measurement.owner: Names the account that owns this measurement, if there is a defined data owner
- instrument.name: The common name of the instrument.
- measurement.latitude: Latitude in degrees
- measurement.longitude: Longitude in degrees
- site.azimuth: Azimuth angle describing the facing of the instrument on its site
- site.campaign: Campaign for this the instrument was at this site (optional)
- site.name: Site name (optional)
- site.region: Site region (optional)
- site.station: Identifier of the station onto which the instrument was installed (optional)

### l1b (Advanced users only)

>Please note that queries that return raw spectral data will return a substantial volume of data on every row, and as there are typically sixty level 1b spectra on just the measurement channels (10 repeats on six channels), it also returns a lot of rows per measurement. The service is written to happily keep sending results to the requesting client until done, but for most practical purposes we strongly recommend limiting the scope of your request to a narrow window of time and location to keep down the size of the results! This is critically important when you view the results in a web browser rather than saving them directly to file, because it is very easy to make a web browser (which wants to cache the file it is displaying) run completely out of memory! Using a tool like curl to pull something from the api straight into a file on disk should be safe.


The request l1b is for advanced users only. Whereas the spectral data that are returned by the GetData request are averaged over 10 spectra and quality screened, the l1b request returns the raw calibrated radiance and irradiance spectra as recorded by the instrument. The return format is fixed, it is not possible to change the columns in the output. However, selecting by date, location and instrument works the same as for the GetData request.
> https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=l1b&TIME=2018-07-11T09:00,2018-07-11T19:00&INSTRUMENT=WISPstation001

A resulting file will look like this:
```
# HEADERLINES 17
# Request processed at 2018-07-19 09:19:31 UTC
# 
# Requested by cnr from 92.111.24.140
# 
# ---original request---
# https://wispcloud.waterinsight.nl/api/query?SERVICE=Data&VERSION=1.0&REQUEST=l1b&TIME=2018-07-11T09:00,2018-07-11T9:30
# 
# ----------------------
# Resource: https://wispcloud.waterinsight.nl/api/query
# Parameters:
#   request ['l1b']
#   time ['2018-07-11t09:00,2018-07-11t9:30']
#   service ['data']
#   version ['1.0']
# ----------------------
# 
measurement.id	measurement.date	measurement.latitude	measurement.longitude	instrument.name	channel.type	channel.position	site.azimuth	channel.orientation	level1.(ir)radiance
	YYYY-MM-DD hh:mm:ss.sssss	degree north	degree east				degree from north	degree from instrument azimuth	W/(m2*nm) if channel.type is Ed else W/(m2*nm*sr) for wavelength [350..900] in 1nm steps
84010	2018-07-11 09:00:05.333859	56.1981	-3.3732	WISPstation003	EdQC	B	337.5	0.0	[0.10583016,0.10906596,...,0.08247125]

84010	2018-07-11 09:00:05.333859	56.1981	-3.3732	WISPstation003	EdQC	B	337.5	0.0	[0.10479023,0.10830143,...,0.08171204]
...
```
The response of the l1b request is different from the GetData request as each row in the GetData response is one measurement, whereas in the l1b response each row is one separate spectrum. This means we have 60 rows for one single measurement (6 channels: 2 each for Ed, Ld and Lu) with 10 repeat measurements each. In practice, there might even be 70 spectra as there is a reference channel. All the rows of each measurement have the same measurement id, they also include a reference to the channel type (channel.type) and channel postition (channel.position). The combination of channel type and channel position uniquely identifies a channel on the instrument in the following way:
- Ed A: Ed at the back 'aft' top of the instrument
- Ed F: Ed at the front 'fore' top of the instrument
- Ld P: Ld channel at the left 'port' front of the instrument
- Ld S: Ld channel at the right 'starboard' front of the instrument
- Lu P: Lu channel at the left 'port' front of the instrument
- Lu S: Lu channel at the right 'starboard' front of the instrument
- (EQC): Reference channel that is not exposed to light, should be ignored

Please note that the radiance channels are "cross-eyed", i.e. they look across the center line of the instrument. So the right "starboar" Lu and Ld channels look at 337.5 degree and the left "port" channels at 22.5 from the instrument orientation, usually north. This information is also provided by the site.azimuth and channel.orientation columns. So if the site.azimuth is 180 (facing south) the Lu P channel (channel.orientation 22.5) will be looking at 202.5 degree from north. It is up to the user to construct a dataset from these individual spectra.